In [1]:
import pandas as pd
import time
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
d = pd.read_csv('ratings.dat', sep='\W+', header=None)


C:\ProgramData\Anaconda3\envs\apriori_env\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [3]:
d.columns = ['userid', 'movieid', 'rating', 'time']

In [4]:
d.head()

,userid,movieid,rating,time
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
d.time

0          978300760
1          978302109
2          978301968
3          978300275
4          978824291
             ...    
1000204    956716541
1000205    956704887
1000206    956704746
1000207    956715648
1000208    956715569
Name: time, Length: 1000209, dtype: int64

In [6]:
d['time'] = [time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(i)) for i in d['time']]

In [7]:
d['time']

0          2000-12-31 22:12:40
1          2000-12-31 22:35:09
2          2000-12-31 22:32:48
3          2000-12-31 22:04:35
4          2001-01-06 23:38:11
                  ...         
1000204    2000-04-26 03:35:41
1000205    2000-04-26 00:21:27
1000206    2000-04-26 00:19:06
1000207    2000-04-26 03:20:48
1000208    2000-04-26 03:19:29
Name: time, Length: 1000209, dtype: object

In [8]:
d['time'] = pd.to_datetime(d['time'], infer_datetime_format=True)

In [9]:
d['time']

0         2000-12-31 22:12:40
1         2000-12-31 22:35:09
2         2000-12-31 22:32:48
3         2000-12-31 22:04:35
4         2001-01-06 23:38:11
                  ...        
1000204   2000-04-26 03:35:41
1000205   2000-04-26 00:21:27
1000206   2000-04-26 00:19:06
1000207   2000-04-26 03:20:48
1000208   2000-04-26 03:19:29
Name: time, Length: 1000209, dtype: datetime64[ns]

In [10]:
df = d.filter(['time', 'movieid'])

In [11]:
df

,time,movieid
0,2000-12-31 22:12:40,1193
1,2000-12-31 22:35:09,661
2,2000-12-31 22:32:48,914
3,2000-12-31 22:04:35,3408
4,2001-01-06 23:38:11,2355
...,...,...
1000204,2000-04-26 03:35:41,1091
1000205,2000-04-26 00:21:27,1094
1000206,2000-04-26 00:19:06,562
1000207,2000-04-26 03:20:48,1096


In [12]:
df.sort_values(by=['time'])

,time,movieid
1000138,2000-04-26 00:05:32,858
1000153,2000-04-26 00:05:54,2384
999873,2000-04-26 00:05:54,593
1000007,2000-04-26 00:06:17,1961
1000192,2000-04-26 00:06:17,2019
...,...,...
825793,2003-02-28 17:45:38,2399
825438,2003-02-28 17:47:23,1407
825724,2003-02-28 17:49:08,3264
825731,2003-02-28 17:49:08,2634


In [13]:
dataset = df.set_index(['time']) 

In [14]:
dataset = dataset.sort_values(by=['time'])

In [15]:
a = set(dataset['movieid'])
print(len(a))

3706


In [16]:
h = {i:[] for i in set(dataset['movieid'])}

In [32]:
# group items into baskets
cache_no = len(set(dataset['movieid']))
group_no = round(cache_no/2)
length = len(dataset)
b = list(range(0,length-1,group_no))
a = list(range(group_no,length,group_no))
for i in range(len(a)):
    l_10 = dataset[b[i]:a[i]]
    for j in h.keys():
        if j in list(l_10.movieid):
            h[j].append(1)
        else:
            h[j].append(0)

In [34]:
new_df = pd.DataFrame.from_dict(h)

In [35]:
new_df

,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,0,1,1,1,1,0,1,0,0,1,...,0,0,0,0,1,1,1,1,0,1
1074,0,1,1,1,1,1,1,0,0,1,...,1,0,0,0,0,1,1,1,1,1
1075,1,1,0,1,0,1,1,0,0,1,...,1,1,0,0,0,1,1,0,0,0
1076,1,0,1,0,1,1,1,0,0,1,...,1,0,0,0,0,1,1,1,0,1


In [36]:
new_df.to_csv('apriori_processed_data.csv', index=False)